In [2]:
import numpy as np
import pandas as pd
import gdal
import os
import glob
import random

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier 

In [7]:
# 从原始训练集生成9+1个子样本。
num_subsamples = 10

#  定义列表来存储原始训练集的子样本（特征和目标）。
x_train_subs = []
y_train_subs = []

# 迭代9（=10-1）次，生成9个较小的训练集
# - `train_size` 必须是0到1之间的浮动值 (exclusive)
# - `random_state=0`: 控制分割，使较小的子样本总是一个子集 
#   较大的子样本
for k in range(num_subsamples - 1):
    x_train_sub, _, y_train_sub, _ = train_test_split(x_train, y_train, 
                                                      train_size=((k+1) / 10),    # 0.1, 0.2, ..., 0.9
                                                      random_state=0)
    
    # 将子样本追加到子样本列表中
    x_train_subs.append(x_train_sub)
    y_train_subs.append(y_train_sub)
    
# 将完整的训练样本追加到子样本的列表中
x_train_subs.append(x_train)
y_train_subs.append(y_train)

NameError: name 'x_train' is not defined

In [8]:
# Helper function #8
def train_and_evaluate_vary_size(estimator, x_train_list, x_test, y_train_list, y_test):
    # Train and evaluate
    # I初始化3个空列表来存储k、验证和测试分数的值 
    # 初始化1个空字典（scores）来存储k：验证分数/测试分数（键：值）对
    K = [] 
    validationf1macro = []
    testf1macro = []
    scores = {}
    
    for k, (x_train, y_train) in enumerate(zip(x_train_list, y_train_list)):
        # 10倍交叉验证
        # 使用10倍交叉验证计算模型的验证分数 
        scores_val = cross_val_score(estimator, x_train, y_train, cv = 10, scoring = 'f1_macro')

        # 计算平均验证分数
        score_val_mean = scores_val.mean()
        
        
        # 培训和测试
        # 对子样本的训练数据进行拟合和训练模型。
        estimator.fit(x_train, y_train)

        # 通过比较实际和预测的结果，计算出测试分数。
        test_predict = estimator.predict(x_test)

        # 通过比较实际和预测的结果，计算出测试分数。
        score_test = metrics.f1_score(y_test, test_predict, average='macro')
        
        
        # 更新名单
        # ---
        key = k + 1
        K.append(key)

        validationf1macro.append(score_val_mean)
        testf1macro.append(score_test)

        # 一个字典保存键：值对，并存储每个k值的验证分数。
        scores[key] = (score_val_mean, score_test)


    # 打印结果
    print("Subsample (%): Macro F1 score (Validation | Test)")

    for key_score in scores:
        score_val = scores[key_score][0]
        score_test = scores[key_score][1]
        print("{:.1f}%: ({:.4f} | {:.4f})".format(key_score*10, score_val, score_test))

In [11]:
# 定义决策树分类器
dtree = DecisionTreeClassifier(criterion = 'gini',splitter = 'best', random_state = 0)

train_and_evaluate_vary_size(dtree, x_train_subs, x_test, y_train_subs, y_test)

NameError: name 'x_test' is not defined

In [13]:
# 辅助功能 #定义一个辅助函数，根据Chi-squared（）方法获得指定数量的特征列表。
def get_chi2_features(X, Y, num_k=10):
    # 创建一个选择器
    # 设置k：我们需要前k个特征
    selector = SelectKBest(chi2, k=num_k)
    
    x_new = selector.fit_transform(X, Y)
    col_index = selector.get_support(indices=True)    # Get the index
    
    cols = X.columns[col_index]                       # Get the name of the feature/column
    
    return cols

In [14]:
cols = get_chi2_features(x_train, y_train, num_k=10)

x_train_chi2 = x_train[cols]
x_test_chi2 = x_test[cols]

x_train_sub_chi2 = []
for k in range(num_subsamples - 1):
    x_train_sub, _, _, _ = train_test_split(x_train_chi2, y_train,           
                                            train_size=((k+1) / 10),
                                            random_state=0)
    x_train_sub_chi2.append(x_train_sub)
#     y_train_subs.append(y_train_sub)
    
x_train_sub_chi2.append(x_train_chi2)
# y_train_subs.append(y_train)

NameError: name 'x_train' is not defined

In [15]:
# 使用前10个特征进行训练和评估 - chi2
dtree = DecisionTreeClassifier(random_state=0)

train_and_evaluate_vary_size(dtree, x_train_sub_chi2, x_test_chi2, y_train_subs, y_test)

NameError: name 'x_train_sub_chi2' is not defined

In [18]:
# #导入袋式分类器的软件包
# 从sklearn.ensemble导入BaggingClassifier

model_bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=0), 
                                  n_estimators=100, random_state=0)

train_and_evaluate_vary_size(model_bagging, x_train_subs, x_test, y_train_subs, y_test)

NameError: name 'x_test' is not defined